# Ulubione słowa posłów

(% jaki stanowi dame słowo w wypodziedzi posła) / (% wszystkich słów użytych w sejmie)

Potrzebujemy:
    
* `all_statements.csv` (np. via `all_statements.rda`)
* `sjp-odm.txt` (http://sjp.pl/slownik/odmiany/)
  
Do zrobienia:
 
* interaktywna wizualizacja
* inne stemowanie?
* wycinanie didaskaliów (np. `(Oklaski na sali)`)?
* słowo per wypowiedź liczone tylko raz? (podobnie...)
* dalszy plan: może pełna analiza tematów / LDA?

Pytani i pomysły do: [Piotr Migdał](http://migdal.wikidot.com/)

In [1]:
import pandas as pd

In [2]:
# słownik stemizacyjny;
stemd = {}

stemf = open("./sjp-odm.txt", encoding="cp1250")
for i, line in enumerate(stemf.readlines()):
    if (i % 10000 == 0):
        print(i, end=" ")
    odmiany = line.strip().split(", ")
    for slowo in odmiany:
        stemd[slowo] = odmiany[0]

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 

In [3]:
# plik z wypowiedziami
sts = pd.read_csv("./all_statements.csv")

In [4]:
sts.head()

,Unnamed: 0,id_statement,surname_name,date_statement,statement
0,1,10.1.001,Sekretarz Poseł Jarosław Górczyński,2012-03-14,"Informuję, że w dniu dzisiejszym odbędą się po..."
1,2,10.1.002,Poseł Sprawozdawca Tadeusz Aziewicz,2012-03-14,Pani Marszałek! Wysoka Izbo! Panie Ministrze!...
2,3,10.1.003,Poseł Adam Szejnfeld,2012-03-14,Pani Marszałkini! Szanowni Państwo! Wysoka Iz...
3,4,10.1.004,Poseł Wojciech Zubowski,2012-03-14,Dziękuję. Pani Marszałek! Wysoka Izbo! Panie ...
4,5,10.1.005,Poseł Michał Tomasz Pacholski,2012-03-14,Pani Marszałek! Wysoka Izbo! W imieniu Klubu ...


In [5]:
# liczenia wszystkich stemizowanych słowa
all_w = {}

# liczenia wszystkich stemizowanych słowa per poseł
by_posel_w = {}

for k, v in sts.iterrows():
    if not isinstance(v["statement"], str):
        continue
        
    posel = v["surname_name"]
    
    if posel not in by_posel_w:
        by_posel_w[posel] = {}
    
    uzyte_slowa = {}
    
    for word in v["statement"].replace(",", "").replace(".", "").replace("?", "").split():
        if word in stemd:
            word = stemd[word]
        elif word.lower() in stemd:
            word = stemd[word.lower()]
            
        ## wariant z liczeniem słowa tylko raz na wypowiedź
        # if word in uzyte_slowa:
        #    continue
        # else:
        #    uzyte_slowa[word] = True
            
        if not word in all_w:
            all_w[word] = 1
        else:
            all_w[word] += 1

        if not word in by_posel_w[posel]:
            by_posel_w[posel][word] = 1
        else:
            by_posel_w[posel][word] += 1

In [6]:
# najczęstsze słowa
sorted([(v, k) for k, v in all_w.items()], reverse=True)[:20]

[(395208, 'w'),
 (336217, 'być'),
 (247408, 'i'),
 (245004, 'on'),
 (218951, 'ten'),
 (192207, 'na'),
 (189321, 'to'),
 (186203, 'z'),
 (159450, 'do'),
 (158116, 'że'),
 (157122, 'się'),
 (150258, 'o'),
 (134686, 'który'),
 (70186, 'pan'),
 (69201, 'ustawa'),
 (66132, 'czy'),
 (65471, '-'),
 (64982, 'tym'),
 (62498, 'a'),
 (62094, 'mieć')]

In [7]:
# uwaga - niektóre podobne słowa de facto są innymi częściami mowy
[(k, v) for k, v in all_w.items() if k.startswith("homos")]

[('homoseksualista', 27),
 ('homoseksualnych)', 1),
 ('homoseksualność', 6),
 ('homoseksualności', 5),
 ('homoseksualizm', 22),
 ('homoseksualny', 112)]

In [8]:
# liczba różnych słów
len(all_w)

95859

In [9]:
wiecej_niz_sto = {word for word, count in all_w.items() if count > 100}

In [10]:
suma_slow = sum(all_w.values())
print(suma_slow)

12880124


In [11]:
# liczba różnuch słów z co najmnijej 100 wystąpień
len(wiecej_niz_sto)

6458

In [12]:
# porównujemy ile razy częściej używa poseł danego słowa niż wszyscy posłowie
czeste = []
for posel, slowa_posla in by_posel_w.items():
    liczba_slow_posla = sum(slowa_posla.values())
    for slowo, uzycie_posla in slowa_posla.items():
        if slowo in wiecej_niz_sto:
            czeste.append({"posel": posel,
                           "slowo": slowo,
                           "uzycia_posla": uzycie_posla,
                           "uzycia_wszystkie": all_w[slowo],
                           "wzglednie": (uzycie_posla/liczba_slow_posla)/(all_w[slowo]/suma_slow)})
            
czeste_df = pd.DataFrame(czeste)

In [13]:
czeste_df.head()

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
0,Wiceprezes Rady Ministrów Minister Finansów Ja...,równoległy,1,353,7.014149
1,Wiceprezes Rady Ministrów Minister Finansów Ja...,3,1,5126,0.483027
2,Wiceprezes Rady Ministrów Minister Finansów Ja...,szansa,1,3624,0.683221
3,Wiceprezes Rady Ministrów Minister Finansów Ja...,wojskowy,4,1895,5.226374
4,Wiceprezes Rady Ministrów Minister Finansów Ja...,zgromadzić,2,1699,2.914649


In [14]:
# wybieramy tylko co najmmniej 2x częsciej używane, z co najmniej 3 wystąpieniami
# i sortujemy po względnej częstotliwości 
czeste_df2 = czeste_df \
  .query("uzycia_posla >= 3") \
  .query("wzglednie >= 3") \
  .sort(columns=["wzglednie"], ascending=False)

In [36]:
czeste_df2.head()

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1260839,Sekretarz Poseł Marzena Okła-Drewnowicz,ślubować,454,524,3817.823041
1260844,Sekretarz Poseł Marzena Okła-Drewnowicz,dopomóc,379,468,3568.490835
912310,Poseł Andrzeja Szlachta,Łukasz,8,121,2150.450622
927985,Poseł Piotr Chmielowski,Pendolino,8,224,2053.591199
708339,Poseó Józef Rojek,pochówek,6,149,1899.863412


In [37]:
czeste_df2.to_csv("czeste_slowa.csv", index=False)

A teraz wreszcie coś ciekawego -  śledzimy zarówno posłów, jak i słowa.

In [16]:
czeste_df2.query("posel == 'Sekretarz Poseł Marzena Okła-Drewnowicz'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1260839,Sekretarz Poseł Marzena Okła-Drewnowicz,ślubować,454,524,3817.823041
1260844,Sekretarz Poseł Marzena Okła-Drewnowicz,dopomóc,379,468,3568.490835
1260878,Sekretarz Poseł Marzena Okła-Drewnowicz,Bóg,379,1128,1480.544070
1260874,Sekretarz Poseł Marzena Okła-Drewnowicz,mi,379,6262,266.696536
1260838,Sekretarz Poseł Marzena Okła-Drewnowicz,Małgorzata,10,276,159.654861
1260760,Sekretarz Poseł Marzena Okła-Drewnowicz,Joanna,4,112,157.374078
1260823,Sekretarz Poseł Marzena Okła-Drewnowicz,Łukasz,4,121,145.668568
1260789,Sekretarz Poseł Marzena Okła-Drewnowicz,Mariusz,6,189,139.888069
1260856,Sekretarz Poseł Marzena Okła-Drewnowicz,Nowak:,3,101,130.885371
1260802,Sekretarz Poseł Marzena Okła-Drewnowicz,Jacek,13,440,130.191282


In [17]:
czeste_df2.query("posel == 'Poseł Andrzej Duda'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
811396,Poseł Andrzej Duda,okrucieństwo,25,112,66.453118
810639,Poseł Andrzej Duda,gwałt,48,217,65.852896
812229,Poseł Andrzej Duda,pobić,28,149,55.945497
813203,Poseł Andrzej Duda,bon,22,153,42.807969
813275,Poseł Andrzej Duda,zgwałcić,19,181,31.251322
813104,Poseł Andrzej Duda,zabójstwo,19,241,23.470910
810386,Poseł Andrzej Duda,P,14,181,23.027290
811871,Poseł Andrzej Duda,referendarz,8,111,21.456574
813185,Poseł Andrzej Duda,trumna,8,112,21.264998
812004,Poseł Andrzej Duda,demonstracja,10,164,18.153047


In [18]:
czeste_df2.query("posel == 'Poseł Radosław Sikorski'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
883571,Poseł Radosław Sikorski,(Wesołość,3,1216,87.538903
883531,Poseł Radosław Sikorski,odnosić,3,3310,32.159307
883675,Poseł Radosław Sikorski,rad,5,8572,20.696707
883641,Poseł Radosław Sikorski,jako,4,14117,10.053817
883673,Poseł Radosław Sikorski,marszałek,3,10705,9.943700
883528,Poseł Radosław Sikorski,poseł,6,25371,8.391258
883547,Poseł Radosław Sikorski,strona,3,13470,7.902547
883638,Poseł Radosław Sikorski,wiedzieć,3,14871,7.158046
883579,Poseł Radosław Sikorski,jeśli,3,16189,6.575286
883651,Poseł Radosław Sikorski,aby,3,17311,6.149114


In [19]:
czeste_df2.query("posel == 'Poseł Antoni Macierewicz'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1178761,Poseł Antoni Macierewicz,wrak,17,105,122.071850
1178376,Poseł Antoni Macierewicz,Antoni,29,379,57.691880
1177523,Poseł Antoni Macierewicz,smoleński,24,355,50.972835
1177362,Poseł Antoni Macierewicz,niesłychany,7,130,40.598556
1178215,Poseł Antoni Macierewicz,kontrwywiad,6,115,39.337732
1178474,Poseł Antoni Macierewicz,Putin,8,160,37.698659
1178750,Poseł Antoni Macierewicz,Mark,15,304,37.202624
1178797,Poseł Antoni Macierewicz,cios,5,107,35.232392
1179015,Poseł Antoni Macierewicz,Niesiołowski:,23,493,35.175220
1177794,Poseł Antoni Macierewicz,Sikorski,34,868,29.533512


In [20]:
czeste_df2.query("posel == 'Poseł Jarosław Kaczyński'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
412562,Poseł Jarosław Kaczyński,niesłychany,14,130,42.117273
413018,Poseł Jarosław Kaczyński,niebywały,13,142,35.803919
414967,Poseł Jarosław Kaczyński,Niesiołowski:,35,493,27.764937
414263,Poseł Jarosław Kaczyński,analogia,7,108,25.348359
413120,Poseł Jarosław Kaczyński,łagodny,7,119,23.005233
414339,Poseł Jarosław Kaczyński,przeciwstawiać,6,106,22.137111
413077,Poseł Jarosław Kaczyński,Piotrowska:,10,178,21.971290
413867,Poseł Jarosław Kaczyński,taśma,10,202,19.360840
414608,Poseł Jarosław Kaczyński,marny,5,102,19.171028
412036,Poseł Jarosław Kaczyński,nieustanny,9,186,18.923660


In [21]:
czeste_df2.query("posel == 'Poseł Ludwik Dorn'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
504875,Poseł Ludwik Dorn,niesłychany,28,130,50.089023
506355,Poseł Ludwik Dorn,Ludwik,20,129,36.055222
505335,Poseł Ludwik Dorn,najsilniejszy,14,102,31.919476
506943,Poseł Ludwik Dorn,wróg,21,196,24.916734
506704,Poseł Ludwik Dorn,współdziałać,30,365,19.114207
508045,Poseł Ludwik Dorn,owszem,19,270,16.365064
507131,Poseł Ludwik Dorn,pragmatyczny,7,105,15.503745
505340,Poseł Ludwik Dorn,blokada,10,156,14.907447
506112,Poseł Ludwik Dorn,sojusznik,14,237,13.737496
507654,Poseł Ludwik Dorn,przegrany,7,123,13.234905


In [22]:
czeste_df2.query("posel == 'Poseł Janusz Palikot'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
360982,Poseł Janusz Palikot,taśma,18,202,35.892480
360926,Poseł Janusz Palikot,marża,8,114,28.266202
360743,Poseł Janusz Palikot,głębszy,15,233,25.930904
360193,Poseł Janusz Palikot,Kaczyński,61,1120,21.937854
362481,Poseł Janusz Palikot,Rosati,7,141,19.996835
361807,Poseł Janusz Palikot,marszałkini,8,163,19.769000
360360,Poseł Janusz Palikot,uszanować,6,124,19.490002
362083,Poseł Janusz Palikot,światopoglądowy,5,104,19.365066
360987,Poseł Janusz Palikot,lęk,5,108,18.647842
362451,Poseł Janusz Palikot,zbieg,5,115,17.512756


In [23]:
czeste_df2.query("posel == 'Poseł Przemysław Wipler'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1265326,Poseł Przemysław Wipler,Przemysław,23,101,79.539471
1267210,Poseł Przemysław Wipler,wyjąć,15,116,45.165779
1267140,Poseł Przemysław Wipler,aparat,44,401,38.325210
1265336,Poseł Przemysław Wipler,przegrywać,10,104,33.584810
1267433,Poseł Przemysław Wipler,rentowy,26,377,24.088416
1265435,Poseł Przemysław Wipler,PIT,29,437,23.178899
1265899,Poseł Przemysław Wipler,kieszeń,42,637,23.029584
1265575,Poseł Przemysław Wipler,obiecywać,28,519,18.843732
1267552,Poseł Przemysław Wipler,danina,18,351,17.911899
1265853,Poseł Przemysław Wipler,liniowy,5,101,17.291189


In [24]:
czeste_df2.query("slowo == 'marihuana'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
540905,Poseł Tomasz Kaczmarek,marihuana,3,106,123.528232
48485,Poseł Jacek Kwiatkowski,marihuana,10,106,70.695022
276167,Poseł Rajmund Miller,marihuana,4,106,46.201750
691568,Poseł Artur Górczyński,marihuana,11,106,41.101373
709675,Poseł Anna Zalewska,marihuana,10,106,37.773751
814640,Poseł Andrzej Rozenek,marihuana,3,106,17.317426
862991,Poseł Michał Kabaciński,marihuana,7,106,13.590487
1041867,Poseł Armand Kamil Ryfiński,marihuana,3,106,10.934424
135264,Poseł Beata Kempa,marihuana,6,106,6.895718
571031,Poseł Andrzej Dera,marihuana,4,106,6.043500


In [25]:
czeste_df2.query("slowo == 'Pendolino'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
927985,Poseł Piotr Chmielowski,Pendolino,8,224,2053.591199
533934,Poseł Elżbieta Apolonia Pierzchała,Pendolino,5,224,68.978591
518724,Poseł Stanisław Lamczyk,Pendolino,7,224,49.368806
866674,Poseł Piotr Król,Pendolino,3,224,42.310930
319556,Poseł Bożenna Bukiewicz,Pendolino,4,224,36.930349
916229,Poseł Teresa Świło,Pendolino,3,224,27.702210
456832,Poseł Konstanty Oświęcimski,Pendolino,8,224,26.745999
840110,Poseł Kazimierz Smoliński,Pendolino,5,224,24.636056
39388,Poseł Grzegorz Adam Woźniak,Pendolino,9,224,21.666526
786878,Poseł Zbigniew Chmielowiec,Pendolino,5,224,18.241404


In [26]:
czeste_df2.query("slowo == 'Bóg'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1260878,Sekretarz Poseł Marzena Okła-Drewnowicz,Bóg,379,1128,1480.544070
469975,Sekretarz Poseł Jan Dziedziczak,Bóg,3,1128,18.957194
731863,Minister Zdrowia Marian Zembala,Bóg,3,1128,18.821785
801842,Poseł Czesław Sobierajski,Bóg,21,1128,17.684899
59915,Poseł Czesław Gluza,Bóg,4,1128,12.173294
250590,Poseł Jerzy Fedorowicz,Bóg,6,1128,8.764398
951427,Poseł Stanisław Pięta,Bóg,8,1128,6.081784
277839,Poseł Ewa Malik,Bóg,9,1128,5.128859
447427,Poseł Stanisław Żelichowski,Bóg,10,1128,5.006818
760952,Poseł Andrzej Kania,Bóg,5,1128,4.437145


In [27]:
czeste_df2.query("posel == 'Prezes Rady Ministrów Donald Tusk'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
121401,Prezes Rady Ministrów Donald Tusk,Dorn,45,197,27.237685
121360,Prezes Rady Ministrów Donald Tusk,wotum,66,385,20.441234
120479,Prezes Rady Ministrów Donald Tusk,scenariusz,57,371,18.319974
119584,Prezes Rady Ministrów Donald Tusk,nieufność,44,313,16.762248
121688,Prezes Rady Ministrów Donald Tusk,lokat,25,182,16.379194
119909,Prezes Rady Ministrów Donald Tusk,domyślać,14,111,15.039347
123096,Prezes Rady Ministrów Donald Tusk,odwagi,34,300,13.513927
119184,Prezes Rady Ministrów Donald Tusk,wyobraźnia,17,157,12.911395
120589,Prezes Rady Ministrów Donald Tusk,napięcie,17,166,12.211380
122488,Prezes Rady Ministrów Donald Tusk,traktatowy,11,112,11.711124


In [28]:
czeste_df2.query("slowo == 'homoseksualny'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1306281,Poseł Jarosław Tomasz Jagiełło,homoseksualny,5,112,79.850790
338951,Poseł Anna Elżbieta Sobecka,homoseksualny,6,112,31.963990
528400,Poseł Robert Biedroń,homoseksualny,14,112,28.091138
952321,Poseł Stanisław Pięta,homoseksualny,3,112,22.969595
3946,Poseł Małgorzata Sadurska,homoseksualny,4,112,22.103908
243607,Poseł Tadeusz Woźniak,homoseksualny,14,112,20.555314
766169,Poseł Barbara Bubula,homoseksualny,4,112,19.762187
977677,Poseł Krystyna Pawłowicz,homoseksualny,4,112,14.879652
949768,Poseł Anna Grodzka,homoseksualny,3,112,11.900359
42997,Poseł Józef Rojek,homoseksualny,7,112,6.123641


In [29]:
czeste_df2.query("slowo == 'homoseksualny'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
1306281,Poseł Jarosław Tomasz Jagiełło,homoseksualny,5,112,79.850790
338951,Poseł Anna Elżbieta Sobecka,homoseksualny,6,112,31.963990
528400,Poseł Robert Biedroń,homoseksualny,14,112,28.091138
952321,Poseł Stanisław Pięta,homoseksualny,3,112,22.969595
3946,Poseł Małgorzata Sadurska,homoseksualny,4,112,22.103908
243607,Poseł Tadeusz Woźniak,homoseksualny,14,112,20.555314
766169,Poseł Barbara Bubula,homoseksualny,4,112,19.762187
977677,Poseł Krystyna Pawłowicz,homoseksualny,4,112,14.879652
949768,Poseł Anna Grodzka,homoseksualny,3,112,11.900359
42997,Poseł Józef Rojek,homoseksualny,7,112,6.123641


In [30]:
czeste_df2.query("slowo == 'uchodźca'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
101898,Poseł Przemysław Krysztofiak,uchodźca,4,203,226.199240
1247834,Poseł Beata Bublewicz,uchodźca,18,203,214.273914
15370,Poseł Roman Kaczor,uchodźca,8,203,98.161109
612350,Poseł Sprawozdawca Anna Nemś,uchodźca,3,203,88.205125
38039,Poseł Grzegorz Adam Woźniak,uchodźca,22,203,58.441512
654469,Poseł Małgorzata Gosiewska,uchodźca,3,203,26.032092
1166053,Minister Spraw Zagranicznych Grzegorz Schetyna,uchodźca,10,203,25.493767
1181545,Poseł Krystyna Ozga,uchodźca,9,203,23.105931
1284680,Poseł Krystyna Łybacka,uchodźca,11,203,17.377197
802124,Poseł Czesław Sobierajski,uchodźca,3,203,14.038400


In [31]:
czeste_df2.query("slowo == 'krzyż'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
577823,Poseł Piotr van der Coghen,krzyż,14,294,87.171580
988215,Sekretarz Poseł Marcin Witko,krzyż,3,294,66.311724
801155,Poseł Czesław Sobierajski,krzyż,9,294,29.079542
985889,Poseł Lech Kołakowski,krzyż,3,294,28.319293
240259,Poseł Jadwiga Wiśniewska,krzyż,12,294,28.314717
208410,Poseł Artur Górski,krzyż,6,294,12.267112
685154,Poseł Jerzy Sądel,krzyż,4,294,10.109015
1189687,Poseł Adam Kwiatkowski,krzyż,6,294,9.835354
129308,Poseł Ryszard Kalisz,krzyż,4,294,8.431881
2972,Poseł Małgorzata Sadurska,krzyż,4,294,8.420536


In [32]:
czeste_df2.query("slowo == 'podatek'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
751353,Poseł Sprawozdawca Wojciech Saługa,podatek,16,10469,13.901817
1301755,Poseł Sprawozdawca Genowefa Tokarska,podatek,17,10469,13.476343
1133971,Poseł Sprawozdawca Marcin Święcicki,podatek,53,10469,10.273590
508444,Poseł Sprawozdawca Renata Zaremba,podatek,5,10469,10.134356
874254,Poseł Zbigniew Matuszczak,podatek,233,10469,8.517677
392300,Poseł Sprawozdawca Lucjan Marek Pietrzczyk,podatek,11,10469,8.036472
149073,Poseł Jarosław Charłampowicz,podatek,28,10469,7.970547
738336,Poseł Sprawozdawca Killion Munyama,podatek,47,10469,7.717150
16041,Poseł Michał Jaros,podatek,36,10469,7.641682
621394,Poseł Genowefa Tokarska,podatek,315,10469,7.535884


In [33]:
czeste_df2.query("slowo == 'zasiłek'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
434317,Poseł Sprawozdawca Iwona Guzowska,zasiłek,9,1304,148.160936
374863,Poseł Sprawozdawca Joanna Bobowska,zasiłek,6,1304,46.628147
594918,Poseł Marzena Okła-Drewnowicz,zasiłek,28,1304,43.526453
982981,Poseł Sprawozdawca Krystyna Ozga,zasiłek,3,1304,41.328016
1061819,Poseł Sprawozdawca Agnieszka Hanajczyk,zasiłek,14,1304,38.189323
332426,Poseł Małgorzata Adamczak,zasiłek,5,1304,27.452462
927593,Poseł Halina Olendzka,zasiłek,3,1304,21.740416
860196,Poseł Sprawozdawca Magdalena Kochan,zasiłek,17,1304,13.701814
598215,Poseł Stanisław Szwed,zasiłek,124,1304,13.605833
661702,Poseł Agnieszka Hanajczyk,zasiłek,18,1304,11.606053


In [34]:
czeste_df2.query("slowo == 'ulga'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
149922,Poseł Sprawozdawca Joanna Mucha,ulga,7,1825,97.250424
820443,Poseł Sprawozdawca Janusz Cichoń,ulga,27,1825,68.323865
1012476,Poseł Sprawozdawca Iwona Kozłowska,ulga,3,1825,20.069011
1235971,Poseł Joanna Mucha,ulga,7,1825,16.315461
956071,Poseł Agnieszka Kołacz-Leszczyńska,ulga,15,1825,15.828952
1267648,Poseł Przemysław Wipler,ulga,70,1825,13.397119
1061735,Poseł Sprawozdawca Agnieszka Hanajczyk,ulga,6,1825,11.694453
622709,Poseł Genowefa Tokarska,ulga,70,1825,9.606474
553085,Poseł Monika Wielichowska,ulga,22,1825,9.473292
925874,Poseł Janusz Cichoń,ulga,16,1825,8.515960


In [35]:
# ale, i, bo - nie więcej niż 4
czeste_df2.query("slowo == 'bo'").head(20)

,posel,slowo,uzycia_posla,uzycia_wszystkie,wzglednie
410168,Poseł Jarosław Stawiarski,bo,19,33155,3.737296
549992,Poseł Mieczysław Kasprzak,bo,212,33155,3.399579
339679,Poseł Sprawozdawca Mieczysław Marcin Łuczak,bo,5,33155,3.286651
1131554,Poseł Andrzej Halicki,bo,117,33155,3.217186
